In [1]:
import os 
import time
import numpy as np
import tensorflow as tf
from tensorflow.python.ops import summary_ops_v2
from tensorflow import keras
from tensorflow.keras import datasets,layers,models,optimizers,metrics

C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\envs\tf2\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\envs\tf2\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\jianghaitao1\AppData\Local\Continuum\anaconda3\envs\tf2\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", n

In [2]:
# 0:默认值，打印所有信息 1：屏蔽INFO信息 2：屏蔽INFO信息和WARNING信息 3：屏蔽INFO,WARNING,ERROR信息
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

In [3]:
def mnist_datasets():
    (x_train,y_train),(x_test,y_test) = datasets.mnist.load_data()
    #numpy默认是float64,tf默认是float32
    x_train,x_test = x_train/np.float32(255),x_test/np.float32(255)
    y_train,y_test = y_train.astype(np.int64),y_test.astype(np.int64)
    train_dataset = tf.data.Dataset.from_tensor_slices((x_train,y_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test,y_test))
    return train_dataset,test_dataset

In [4]:
train_ds,test_ds = mnist_datasets()

In [5]:
train_ds = train_ds.shuffle(60000).batch(100)
test_ds = test_ds.batch(100)

In [6]:
# 构建模型
model = tf.keras.Sequential([
    layers.Reshape(target_shape=[28,28,1],input_shape=[28,28]),
    layers.Conv2D(2,5,padding='same',activation=tf.nn.relu),
    layers.MaxPooling2D((2,2),(2,2),padding='same'),
    layers.Conv2D(4,5,padding='same',activation=tf.nn.relu),
    layers.MaxPool2D((2,2),(2,2),padding='same'),
    layers.Flatten(),
    layers.Dense(32,activation=tf.nn.relu),
    layers.Dropout(rate=0.4),
    layers.Dense(10)
])

In [7]:
optimizer = optimizers.SGD(lr=0.01,momentum=0.5)

In [8]:
compute_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [9]:
compute_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [10]:
def train_one_step(model,optimizer,images,labels):
    with tf.GradientTape() as tape:
        logits = model(images,training=True)
        loss = compute_loss(labels,logits)
        compute_accuracy(labels,logits)
    grads = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))
    return loss